# Scraping UK asylum data (18 October 2021)

This notebook scraps the tribunal decision's data related to asylum applications from https://tribunalsdecisions.service.gov.uk/

The script mac_env.sh should be run from the terminal to install the necessary libraries and to activate the environment.

In [12]:
from os import listdir
from os.path import isfile, join, getsize
import numpy as np
import time
import re
import json
import pickle
import pandas as pd
import whois
import sys
import datetime
from tqdm import tqdm
import textract

import sys
IN_COLAB = 'google.colab' in sys.modules


# What environment am I using?
print(f'Current environment: {sys.executable}')

# Change the current working directory
os.chdir('/Users/albertamurgopacheco/Documents/GitHub/TFM')
# What's my working directory?
print(f'Current working directory: {os.getcwd()}')


Current environment: /Users/albertamurgopacheco/anaconda3/envs/tfm/bin/python
Current working directory: /Users/albertamurgopacheco/Documents/GitHub/TFM


In [16]:
# Define working directories in colab and local execution

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')
    docs_path = '/content/gdrive/MyDrive/TFM/data/raw'
    input_path = '/content/gdrive/MyDrive/TFM'
    output_path = '/content/gdrive/MyDrive/TFM/output'

else:
    docs_path = './data/raw'
    input_path = '.'
    output_path = './output'

# WORD DOCUMENTS

In [17]:
# Extract all files in docs_path (name, size and last modification_time)
files_name_list_raw = [f for f in listdir(docs_path) if isfile(join(docs_path, f))]
files_size_list = [getsize(join(docs_path, f)) for f in listdir(docs_path) if isfile(join(docs_path, f))]

In [15]:
# Obtain/check number of files
print('Number of files:', len(files_name_list_raw))
# Unique files based on size file_name
print('Number of unique file names:', len(set(files_name_list_raw)))
# Unique files based on size file_name
print('Number of unique file sizes:', len(set(files_name_list_raw)))

Number of files: 34953
Number of unique file names: 34953
Number of unique file sizes: 34953


There are no duplicated files.

In [ ]:
# Eliminate something from the file name
#files_name_list = [re.sub('^\d*_|\.\w{3}\Z', '', file_name) for file_name in files_name_list_raw]


Create a function to obtain the data from the court's decision detailed page using beautifulSoup.     TO DO: 2. Try with different files make sure it works, 3. Capture exceptions and 204 responses. 4. Create function, 5. How am I storing the dicts? In a list? 5. Try function with just a few obs. https://stackoverflow.com/questions/20638006/convert-list-of-dictionaries-to-a-pandas-dataframe



In [65]:
# Delete DS_Store files in raw data folder
!find . -name '.DS_Store' -type f -delete

# Destination directory of txt files
dest_files_path = os.path.join(os.getcwd(), 'data/processed/txt_files')

# Loop to extract txt from word files (with decorator progress bar)
for word_file in  tqdm(os.listdir(docs_path)):
    file, extension = os.path.splitext(word_file)
    
    # Create txt file concatenating .txt extension to file name
    dest_file_name = file + '.txt'
    
    # Extract text from the file
    content = textract.process(os.path.join(docs_path, word_file))
    
    # Create and open new file & prepare to write the Binary Data (represented by wb - Write Binary)
    write_text_file = open(os.path.join(dest_files_path, dest_file_name), "wb")
    
    # Write the content and close the newly created file
    write_text_file.write(content)
    write_text_file.close()

 47%|████▋     | 16366/34952 [25:02<28:26, 10.89it/s]


ShellError: The command `antiword ./data/raw/HU077022015.doc` failed with exit code 1
------------- stdout -------------
b''------------- stderr -------------
b'./data/raw/HU077022015.doc is not a Word Document.\n'

Now that we have defined all the necessary functions, we can open a browser and start scraping.

  Obtain a dataframe with the link and the date of each of the tribunal decisions. 
  1. GET DATES
 2. Build function that gets data per page
 3. Loop over 1667 pages and append the lists
 4. Build dataframe or iterate over each list to 1) open url 2) extract additional content, and 3) download file using class doc-file




Now, we scrape the detailed data on each tribunal decission.